# RNA-DNAm: scBOND

The following tutorial demonstrate how to use scBOND model making translation between scRNA-seq and scDNAm profiles.

There are three parts of this tutorial:

**(1) Read in data and load it to scBOND.** This part will tell you how to load and pre-process scRNA-seq and scDNAm profiles data for scBOND model.

**(2) Construct a scBOND model.** This part will tell you how to generate and train a scBOND model.

**(3) Get prediction and evaluate the performance.** This part will tell you how to get prediction from scBOND model and evaluate the performance of prediction.

## Read in data and load it to scBOND

scBOND has integrated the process of pre-processing, constructing model, training model and evaluating in scBOND.bond for paired scRNA-seq and scDNAm profiles. You could easily use it as follow:

In [1]:
import scBond
from scBond.bond import Bond
import scanpy as sc

In [2]:
bond = Bond()

Here we use the [**HumanBrainA data**](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE140493) with 4013 nuclei isolated from postmortem human frontal cortex(Luo, C., et al., 2023) for train and the [**HumanBrainB data**](https://datasets.cellxgene.cziscience.com/b8bedb33-4f07-41b4-9656-8c428d549c94.h5ad) compriseing 3142 cells from the human cerebral cortex for test

In [3]:
RNA_data = sc.read_h5ad('adata_rna.h5ad')
MET_data = sc.read_h5ad('adata_met.h5ad')
RNA_test = sc.read_h5ad('test_rna.h5ad')

In [4]:
#ensure that RNA and DNAm data are paired
rna_index_order = RNA_data.obs.index
MET_data = MET_data[rna_index_order, :]

In [5]:
#make the indices of the two RNA datasets var the same, facilitating concatenation
new_index = [idx.split('.')[0] for idx in RNA_data.var.index]
RNA_data.var.index = new_index
RNA_data = RNA_data[:, ~RNA_data.var.index.duplicated(keep='first')]

In [6]:
RNA_data.obs['cell_type'] = RNA_data.obs['MajorType']
RNA_data = RNA_data.concatenate(RNA_test, join='inner')

In [7]:
all_indices = MET_data.var.index.tolist()
MET_data.var['chrom'] = [idx[:4] if len(idx) >= 4 else idx for idx in all_indices]

We provide data split settings in scBOND.split_dataset for reproducibility. You could use parameters train_id, validation_id and test_id customize your own datasets splitting settings.

In [8]:
train_id = [i for i in range(len(MET_data.obs_names))]
test_id = [i+len(MET_data.obs_names) for i in range(len(RNA_test.obs_names))]

Then we load data into scBOND.

In [9]:
bond.load_data(RNA_data, MET_data, train_id, test_id)

[INFO] Bond: successfully load in data with

RNA data:
AnnData object with n_obs × n_vars = 35078 × 54928
    obs: 'fig2_tsne_0', 'fig2_tsne_1', 'fig2_umap_0', 'fig2_umap_1', 'MajorType', 'SubType', 'cell_type', 'roi', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'tissue_type', 'assay', 'disease', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'batch'
    var: 'Biotype-1', 'Chromosome-1', 'End-1', 'Gene-1', 'Start-1', 'feature_is_filtered-1', 'feature_name-1', 'feature_reference-1', 'feature_biotype-1', 'feature_length-1', 'feature_type-1'

MET data:
AnnDa

scBOND could pre-process data Automatically, custom settings is also provided.

In [10]:
bond.data_preprocessing()

[INFO] RNA preprocessing: normalize size factor.
[INFO] RNA preprocessing: log transform RNA data.
[INFO] RNA preprocessing: choose top 3000 genes for following training.
[INFO] MET preprocessing: Filtering features: min 29.0 cells (0.700%) per feature
[INFO] MET preprocessing: Performing missing value imputation using method: median
[INFO] MET preprocessing: Performing min-max scaling.


## Construct a scBOND model

We use no data augmentation strategy here, for more information, see in scBOND-Aug.

In [11]:
bond.augmentation(enable_augmentation=False)

scBOND model needs the information of peaks chromosomes, we should sort the scDNAm profiles data with chromosomes and tell scBOND the count of peaks in each chromosome.

In [12]:
chrom_list = []
last_one = ''
for i in range(len(bond.MET_data_p.var.chrom)):
    temp = bond.MET_data_p.var.chrom[i]
    if temp[0 : 3] == 'chr':
        if not temp == last_one:
            chrom_list.append(1)
            last_one = temp
        else:
            chrom_list[-1] += 1
    else:
        chrom_list[-1] += 1
        
print(chrom_list, end="")

[22471, 23784, 19482, 18767, 17739, 16740, 15537, 14281, 11873, 94526, 12268, 14783, 2012, 1]

In [13]:
sum(chrom_list)

284264

We construct and train scBOND model with default settings.

In [14]:
bond.construct_model(chrom_list=chrom_list)

[INFO] Bond: successfully construct bond model.


In [15]:
bond.train_model()

[INFO] Bond: training bond model ...
[INFO] Trainer: MET pretraining ...
MET pretrain: 100%|█████████████████████| 100/100 [09:49<00:00,  5.90s/it, train=0.2708, val=0.2739]
[INFO] Trainer: RNA pretraining ...
RNA pretrain: 100%|█████████████████████| 100/100 [08:36<00:00,  5.17s/it, train=0.1372, val=0.1445]
[INFO] Trainer: Integrative training ...
Integrative training: 100%|█████████████| 200/200 [31:05<00:00,  9.33s/it, train=1.3556, val=1.3499]


## Get prediction and evaluate the performance

You could get cross-modal predictions using bond.test_model with default settings. We also provided more information metrics in this function, see in API.

In [16]:
R2M_predict = bond.predict_single_modal(data_type='rna', load_model = False)

[INFO] SingleModalPredictor: predicting methylation from RNA data...
RNA to Methylation predicting...: 100%|███████████████████████████| 486/486 [00:20<00:00, 23.81it/s]
